#### SARSAMAX

In [12]:
from graphics import *
import random
import numpy as np
from math import sqrt

#### create 5x5 Grid world

In [2]:
win = GraphWin("My Window", 600,600)
win.setBackground(color_rgb(255,255,255))

In [3]:
def line(x1,y1,x2,y2):
    """
    create a line between pt(x1,y1) and pt(x2,y2) and return it
    """
    ln = Line(Point(x1,y1),Point(x2,y2))
    ln.setOutline(color_rgb(0,0,0))
    ln.setWidth(2)
    
    return ln

In [4]:
#border lines
border1 = line(50,50,550,50)
border1.draw(win)

border2 = line(550,50,550,550)
border2.draw(win)

border3 = line(50,550,550,550)
border3.draw(win)

border4 = line(50,50,50,550)
border4.draw(win)

#lines horizontal
ln1 = line(50,150,550,150)
ln1.draw(win)

ln2 = line(50,250,550,250)
ln2.draw(win)

ln3 = line(50,350,550,350)
ln3.draw(win)

ln4 = line(50,450,550,450)
ln4.draw(win)

#lines vertical

lnv1 = line(150,50,150,550)
lnv1.draw(win)

lnv2 = line(250,50,250,550)
lnv2.draw(win)

lnv3 = line(350,50,350,550)
lnv3.draw(win)

lnv4 = line(450,50,450,550)
lnv4.draw(win)

#create goal, obstacle and hole

rect_goal = Rectangle(Point(450,50), Point(550,150))
rect_goal.setFill(color_rgb(0,255,0))
rect_goal.draw(win)

rect_hole = Rectangle(Point(450,150), Point(550,250))
rect_hole.setFill(color_rgb(255,0,0))
rect_hole.draw(win)

rect_obs1 = Rectangle(Point(150,350), Point(250,450))
rect_obs1.setFill(color_rgb(0,0,0))
rect_obs1.draw(win)

rect_obs2 = Rectangle(Point(150,250), Point(250,350))
rect_obs2.setFill(color_rgb(0,0,0))
rect_obs2.draw(win)

Rectangle(Point(150.0, 250.0), Point(250.0, 350.0))

#### Create agent look like circle

In [5]:
agent = Circle(Point(100,500), 25)
agent.setFill(color_rgb(0,0,255))
agent.draw(win)

Circle(Point(100.0, 500.0), 25)

In [6]:
def up(agent):
    val = agent.getCenter()
    
    if (val.getY()) == 100 or (val.getY() == 500 and val.getX() == 200) :
        pass
    else :
        agent.move(0,-100)
        
    
def down(agent):
    val = agent.getCenter()
    
    if val.getY() == 500 or (val.getX() == 200 and val.getY() == 200):
        pass
    else :
        agent.move(0,100)

def right(agent):
    val = agent.getCenter()
    
    if val.getX() == 500 or (val.getX() == 100 and (val.getY() == 300 or val.getY() == 400)):
        pass
    else :
        agent.move(100,0)

def left(agent):
    val = agent.getCenter()
    
    if val.getX() == 100 or (val.getX() == 300 and (val.getY() == 300 or val.getY() == 400)):
        pass
    else:
        agent.move(-100,0)
        
def action(agent,val):
    if val == 0:
        up(agent)
    elif val == 1:
        down(agent)
    elif val == 2:
        right(agent)
    else :
        left(agent)

#### Reward for agent in the each state

In [7]:
reward = np.array([[-1,-1,-1,-1,-1],
                   [-1,-20,-1,-1,-1],
                   [-1,-20,-1,-1,-1],
                   [-1,-1,-1,-1,-100],
                   [-1,-1,-1,-1,100]])

In [8]:
def rewards(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100) 
    
    
    return reward[j][i]

def observation(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100) 
    
    return [i,j]

def env_close():
    win.close()

def reset(agent):
    agent.move(-400,400)

def set_agent(agent,obs):
    pos = agent.getCenter()
    
    pos = [pos.getX(), pos.getY()]
    
    refer_pos = [(obs[0]+1) * 100, (500 - (obs[1]*100))]
    
    
    agent.move(refer_pos[0]-pos[0], refer_pos[1] - pos[1])

#### Create the value function and policy

In [22]:
Q = np.zeros([5,5,4])

In [49]:
Q[4,4] = [100.,100.,100.,100.]

#### Declare the variable

In [23]:
episode = 300
max_episode_length = 50
learning_rate = .01
discount_factor = 0.9

In [24]:
for j in range(episode):  
    epsilon = 1 / (sqrt(j)+1)
    
    S = observation(agent)
    A = np.argmax(Q[S[0],S[1]])
    
    for i in range(max_episode_length):
        action(agent,A)
        
        R = rewards(agent)
        
        S_new = observation(agent)
        
        p = random.uniform(0,1)
        
        if p > epsilon:
            A_new = np.argmax(Q[S_new[0],S_new[1]])
        else :
            A_new = np.random.choice([0,1,2,3])
        
        #add probability to each action
        if A_new == 0:
            A_new = np.random.choice([0,1,2,3],p=[0.8,0,0.1,0.1])
        elif A_new == 1:
            A_new = np.random.choice([0,1,2,3],p=[0,0.8,0.1,0.1])
        elif A_new == 2:
            A_new = np.random.choice([0,1,2,3],p=[0.1,0.1,0.8,0])
        elif A_new == 3:
            A_new = np.random.choice([0,1,2,3],p=[0.1,0.1,0, 0.8])
            
       
        #temporal difference learning
    
        Q[S[0],S[1],A] = Q[S[0],S[1],A] + (learning_rate * (R + (discount_factor * np.max(Q[S_new[0],S_new[1]])) - Q[S[0],S[1],A]))
 
        S = S_new
        A = A_new
        
        if (S[0] == 4) and (S[1] == 4):
            reset(agent)
            S = observation(agent)
            A = np.argmax(Q[S[0],S[1]])
            continue
        

In [25]:
Q

array([[[ 3.07461969e+01, -2.22834582e+00, -2.22366183e+00,
         -2.21730565e+00],
        [ 4.04343534e+01, -6.89566805e-01,  1.18791152e+01,
          1.26555363e+01],
        [ 5.01795922e+01,  1.30249883e+00,  1.98198841e+01,
          1.87701356e+01],
        [ 3.74899370e+00,  1.65442099e+01,  5.94337274e+01,
          5.33127075e+00],
        [ 3.47587155e-01,  3.22496566e+00,  1.87917044e+01,
         -2.36575375e-01]],

       [[-1.68895286e+00, -1.67354197e+00, -1.58640228e+00,
         -1.65701294e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.96698821e+01,  3.23706875e+01,  6.91410946e+01,
          2.69705300e+00],
        [ 7.09831124e+00,  7.86432677e+00,  5.98753448e+01,
          6.37576567e-02]],

       [[-3.00877215e-01, -9.70335374e-01, -9.97451284e-01,
         -1.10221974e+00],
        [ 3.68739166e+00, -4.90674182e-

In [ ]:
for exp in experience:
    set_agent(agent,[exp[0],exp[1]])
    action(agent,exp[2])

In [26]:
for i in range(5):
    for j in range(5):
        print(np.argmax(Q[i,j]), end= " ")
    print()

0 0 0 2 2 
2 0 0 2 2 
0 0 0 0 2 
0 0 0 0 2 
3 3 3 0 0 


#### After 300 episode with 50 step in each episode using SARSAMAX

| right| right| right| right|  GOAL  |
|------|------|------|------|--------|
| right| right|  up  |  up  |   up   |
|------|------|------|------|--------|
|  up  |      |  up  |  up  |  left  |
|------|------|------|------|--------|
|  up  |      |  up  |  up  |  left  |
|------|------|------|------|--------|
|  up  | right|  up  |  up  |  left  |

#### After 1000 episode with 10 step in each episode using SARSA

| right| right| right| right|  GOAL  |
|------|------|------|------|--------|
| right| right|  up  |  up  |   up   |
|------|------|------|------|--------|
|  up  |      |  up  |  up  |  left  |
|------|------|------|------|--------|
|  up  |      |  up  |  up  |  left  |
|------|------|------|------|--------|
|  up  | right|  up  | right|  right |

In [27]:
win.close()

#### Compare both SARSA and SARSAMAX beacuse of exploratory behaviour in SARSAMAX make converge better policy